In [ ]:
import pandas as pd
data = pd.read_csv('',delimiter='\t')
data

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
punctuations = '!\"#$%&\'()*+,-/:;<=>?@[\]^_`{|}~'

In [ ]:
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() for word in doc ]
    mytokens = [ word for word in mytokens if word not in punctuations ]
    sentence = " ".join(mytokens)
    return sentence

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
from sentence_transformers.datasets import SentenceLabelDataset
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
import logging
import os
import random
from collections import defaultdict

In [ ]:
import nltk
from nltk import tokenize

def single_sentences(data):
    data['cleaned'] = data['text'].apply(spacy_tokenizer)
    data=data.assign(scombi="")

    for c in range(len(data)):
        try:
            sentence_tok = []
            sentence_tok = tokenize.sent_tokenize(data["cleaned"][c])
            if sentence_tok[-1].endswith('.'):
                sentence_tok =  sentence_tok
            else:
                sentence_tok[-1] = str(sentence_tok[-1]) + str('.')
            data.loc[c, "scombi"] = '  '.join(sentence_tok)
        except:
            print('1')
    list_sent = []
    list_cat = []
    for c in range(len(data)):
        token = data.scombi[c].split('  ')
        list_sent.extend(token)
        list_temp_cat = [data.Category[c]]*len(token)
        list_cat.extend(list_temp_cat)
        
    return list_sent,list_cat

In [ ]:
P = single_sentences(data)

In [ ]:
df_list_comp = pd.DataFrame(list(zip(P[0],P[1])),columns = ['text','Category'])

In [ ]:
data = df_list_comp

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)
data

In [ ]:
classes=data['Category'].unique().tolist()
nb_classes = len(classes)
print(nb_classes)
print(classes)

In [ ]:
for i,type_c in enumerate(classes):
   for j,type_t in enumerate(data['Category']):
       if type_c == type_t :
           data.loc[j,'Category'] = i

In [ ]:
import random
lst = list(range(len(data)))
random_list = random.sample(lst,5000)

In [ ]:
data= data.assign(split="train")

for z in random_list:
    data.loc[z,'split'] = 'eval'
data.head(20)

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
import pandas as pd

train_samples=[]
dev_samples=[]
for i in range(len(data)):
    if data['split'][i] == 'train':
        train_samples.append(InputExample(texts=[data['text'][i]],label=data['Category'][i]))
    else:
        dev_samples.append(InputExample(texts=[data['text'][i]],label=data['Category'][i]))

In [ ]:
def triplets_from_labeled_dataset(input_examples):
    # Create triplets for a [(label, sentence), (label, sentence)...] dataset
    # by using each example as an anchor and selecting randomly a
    # positive instance with the same label and a negative instance with a different label
    triplets = []
    label2sentence = defaultdict(list)
    for inp_example in input_examples:
        label2sentence[inp_example.label].append(inp_example)

    for inp_example in input_examples:
        anchor = inp_example

        if len(label2sentence[inp_example.label]) < 2:  # We need at least 2 examples per label to create a triplet
            continue

        positive = None
        while positive is None or positive.guid == anchor.guid:
            positive = random.choice(label2sentence[inp_example.label])

        negative = None
        while negative is None or negative.label == anchor.label:
            negative = random.choice(input_examples)

        triplets.append(InputExample(texts=[anchor.texts[0], positive.texts[0], negative.texts[0]]))

    return triplets
random.seed(42) 
dev_triplets = triplets_from_labeled_dataset(dev_samples)

In [ ]:
from torch.utils.data import DataLoader
train_data_sampler = SentenceLabelDataset(train_samples)
train_dataloader = DataLoader(train_data_sampler, batch_size=16, drop_last=True)

In [ ]:
dev_evaluator = TripletEvaluator.from_input_examples(dev_set, name="medical_eval")
logging.info("Performance before fine-tuning:")
dev_evaluator(model)

In [ ]:
from torch.utils.data import DataLoader
batch_size=16
loader = DataLoader(train_samples,batch_size,shuffle=True)
#evaluator


In [ ]:
from sentence_transformers import models, SentenceTransformer

#bert_bio = models.Transformer('dmis-lab/biobert-v1.1')
#pooler = models.Pooling(bert_bio.get_word_embedding_dimension(),pooling_mode_mean_tokens = True)
#model = SentenceTransformer(modules=[bert_bio, pooler])

model= SentenceTransformer('all-MiniLM-L6-v2',device='cuda')
model

In [ ]:
from sentence_transformers import losses

loss = losses.BatchAllTripletLoss(model = model,margin=0.1)

In [ ]:
model.encode(data['text'][0])

In [ ]:
epochs = 1
warmup_steps = int(len(loader) * epochs * 0.1)
model.fit(train_objectives=[(loader, loss)],
          
          epochs = epochs,
          warmup_steps = warmup_steps,
          output_path='./triple_loss_all_1000'
          )

In [ ]:
test_evaluator = TripletEvaluator.from_input_examples(test_set, name="trec-test")
model.evaluate(test_evaluator)

In [ ]:
import pickle
filename = 'tripleloss_all_1000_1_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
data = pd.read_pickle('/kaggle/input/dataset/translated_full')
data = data.sample(frac=1).reset_index(drop=True)
data_test = pd.read_csv('/kaggle/input/dataset/testing.txt')
data_test = data_test.drop_duplicates()
data_test = data_test.reset_index(drop =True)
data

In [ ]:
classes=data['Category'].unique().tolist()
nb_classes = len(classes)
print(nb_classes)
print(classes)

In [ ]:
for i,type_c in enumerate(classes):
   for j,type_t in enumerate(data['Category']):
       if type_c == type_t :
           data.loc[j,'Category'] = i
for i,type_c in enumerate(classes):
   for j,type_t in enumerate(data_test['Category']):
       if type_c == type_t :
           data_test.loc[j,'Category'] = i

In [ ]:
data['embeddings'] = data['text'].apply(model.encode)
data_test['embeddings'] = data_test['text'].apply(model.encode)

In [ ]:
from sklearn.model_selection import train_test_split
X_train = data['embeddings'].to_list()
y_train = data['Category'].to_list()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)
X_test = data_test['embeddings'].to_list()
y_test = data_test['Category'].to_list()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

clfs = [
    ('LogisticRegression', LogisticRegression(max_iter=3000,
                                              class_weight='balanced')
    ),
    ('RandomForest', RandomForestClassifier(max_depth=18,
                                            n_estimators=75,
                                            random_state=0)
    ),
    ('KNN 5', KNeighborsClassifier(n_neighbors=4)
    ),
    ('SVM C1', SVC(C=1,
                   class_weight='balanced')
    )]

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score


def print_val_scores(scores: list[float]) -> None:

  print(f'Cross validation scores: mean: {np.mean(scores):.3f}, '
        f'all: {[round(score, 3) for score in scores]}')


def print_stratified_kfold(clfs: list[tuple[str, any]], X_train: pd.DataFrame,
                           y_train: pd.Series, n_splits: int = 5, cv: int = 5,
                           ) -> None:

  for clf in clfs:
    print(f'\nStratifiedKFold - classifier: {clf[0]}:\n')
    skf = StratifiedKFold(n_splits=n_splits)

    scores = cross_val_score(clf[1],
                            X_train,
                            y_train,
                            cv=cv)

    print_val_scores(scores)

In [ ]:
print_stratified_kfold(clfs, X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

clf =  SVC(C=1,class_weight='balanced')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# prob = clf.predict_proba(X_test)
# print(prob)

accuracy = np.mean(y_pred == y_test)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title(f'SVM - acc {accuracy:.3f}', size=15)
plt.show()